# Stratified Propensity Score Matching Baseline 

### Propensity Score Model 
- Input: Same embeddings used in matching 
- Model: 
    - Logistic regression P(T|E) where T is binary treatment and E is the embeddings
    - Cross-fitting with cross validation for the regularizer 
- Ouput: Inference scores for each document that are the propensity scores 
    
### Causal Estimator: Stratified Propensity Score Matching  
- Input: Inference scores for each doc (above) 
- Approach: 5 strata, matching causal estimator 
- Output: Causal ATE 

In [16]:
# imports 
import numpy as np 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold, StratifiedShuffleSplit
from sklearn.calibration import calibration_curve
from copy import deepcopy
import pprint

from sklearn.metrics import (
    f1_score,
    accuracy_score,
    mean_squared_error,
    r2_score,
    roc_auc_score,
    average_precision_score,
)

In [2]:
# create synthetic toy data just to make sure code runs 
# Raymond--you'll replace this with the real data 
nT1 = 100 #number of T=1 units
nT0 = 300 #number of T=0 units 
n_total = nT1 + nT0
T = np.array([1]*nT1 + [0]*nT0) #array with treatment values 
np.mean(T)

0.25

In [3]:
embedding_dim = 50 #dimensionality of embeddings 
E = np.random.randn(n_total, embedding_dim)
E.shape

(400, 50)

In [32]:
docids = np.arange(n_total)

In [33]:
assert E.shape[0] == T.shape[0] == n_total == docids.shape[0]

### Propensity Score Model 

In [5]:
NUM_CROSSFIT_SPLITS = 2
NUM_CROSSVAL_SPLITS = 4 
RANDOM_STATE = 42 #for replication 

CLASSIFIER = Pipeline([(
            "model",
            LogisticRegression(
                l1_ratio=0.1,
                solver="saga",
                max_iter=20000,
                tol=1e-3, 
                penalty="elasticnet",
                dual=False,
                class_weight="balanced",
                random_state=42,
                ),),])

CLASSIFIER_GRID = {
    "model__C": [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1],
}

In [7]:
#Note: May take awhile to run
prob_t_training = np.array([np.nan]*n_total) #only for reporting training acc numbers 
prob_t_inference = np.array([np.nan]*n_total) #array to save predicted propensity scores 
# Crossfitting 
crossfit_split = list(StratifiedKFold(n_splits=NUM_CROSSFIT_SPLITS, shuffle=True, random_state=RANDOM_STATE).split(E, T))
for crossfit_number, (train_inds, test_inds) in enumerate(crossfit_split):
    print('Crossfit num=', crossfit_number)
    # Training 
    # Cross validation for the training split 
    inner_cv = StratifiedShuffleSplit(
                n_splits=NUM_CROSSVAL_SPLITS,
                test_size=1/NUM_CROSSVAL_SPLITS,
                random_state= RANDOM_STATE,
            )
    
    t_model_gridsearch = GridSearchCV(
                        estimator=deepcopy(CLASSIFIER),
                        param_grid=deepcopy(CLASSIFIER_GRID),
                        cv=inner_cv,
                        scoring="roc_auc",
                        refit=True,
                    )
    
    t_model_gridsearch.fit(E[train_inds], T[train_inds])
    prob_t_training[train_inds] = t_model_gridsearch.predict_proba(E[train_inds])[:, 1]
        
    # Inference
    # Probability of T=1
    prop_t = t_model_gridsearch.predict_proba(E[test_inds])[:, 1]
    prob_t_inference[test_inds] = prop_t

Crossfit num= 0


/Users/katherinekeith/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/katherinekeith/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/katherinekeith/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Crossfit num= 1


/Users/katherinekeith/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
# make sure all items get inference predictions 
assert np.mean(np.isnan(prob_t_training)) == 0.0
assert np.mean(np.isnan(prob_t_inference)) == 0.0

In [17]:
# Training metrics 
pred_model_report_out = {}

def mean_predictions(dummy, y_pred):
    """
    Helpful for error diagnosing. Returns the mean of the predcted values
    Args:
        - dummy : we need this arg so that this function looks the same as
        sklearn error metrics that take inputs y_true, y_pred
        - y_pred : np.array
    """
    return np.mean(y_pred)

def calibration_rmse(y_true, y_pred): 
    """
    Calculates calibration root mean squared error (RMSE). 
    Calibration is the extent to which a model's probabilistic predictions match their 
    corresponding empirical frequencies. 
    See Nguyen and O'Connor 2015's introduction and definitions 
    https://www.emnlp2015.org/proceedings/EMNLP/pdf/EMNLP182.pdf
    """
    prob_true, prob_pred = calibration_curve(y_true, y_pred, n_bins=10, strategy='uniform')
    rms = mean_squared_error(prob_true, prob_pred, squared=False) #False returns RMSE vs MSE 
    return rms


def mean_truth(y_true, dummy):
    """
    Helpful for error diagnosing. Returns the mean of the true values
    Args:
        - y_true : np.array
        - dummy : we need this arg so that this function looks the same as
        sklearn error metrics that take inputs y_true, y_pred
    """
    return np.mean(y_true)

# these classification metrics need the "hard" predictions, e.g. y=[0, 0, 1, ...]
class_hard_name2metric_func = {
    "f1": f1_score,
    "acc": accuracy_score,
    "mean_hard_pred": mean_predictions,
    "mean_true": mean_truth,  # should be same for hard or soft
}

# these classification metrics need the "score" predictions, e.g. y=[0.6, 0.77, 0.2, ...]
class_scores_name2metric_func = {
    "roc_auc": roc_auc_score,
    "ave_prec": average_precision_score,
    "calibration_rmse": calibration_rmse,
    "mean_soft_pred": mean_predictions,
    "mean_true": mean_truth,  # should be same for hard or soft
}

#hard classifications
t_pred_hard = (prob_t_training > 0.5).astype(int)

assert t_pred_hard.shape == prob_t_training.shape == T.shape

str_sep = "--"
for metric_str, metric_func in class_hard_name2metric_func.items():
    pred_model_report_out["treatment_model" + str_sep + metric_str] = metric_func(T, t_pred_hard)
for metric_str, metric_func in class_scores_name2metric_func.items():
    pred_model_report_out["treatment_model" + str_sep + metric_str] = metric_func(T, prob_t_training)

pprint.pprint(pred_model_report_out)

#TODO: Could do this for cross-validation splits as well 

{'treatment_model--acc': 0.705,
 'treatment_model--ave_prec': 0.5062102030384338,
 'treatment_model--calibration_rmse': 0.20752614108345802,
 'treatment_model--f1': 0.5530303030303031,
 'treatment_model--mean_hard_pred': 0.41,
 'treatment_model--mean_soft_pred': 0.4546036283311568,
 'treatment_model--mean_true': 0.25,
 'treatment_model--roc_auc': 0.7799}


### Causal Estimator: Stratified Propensity Score Matching  

In [24]:
NUM_STRATA = 5 

In [34]:
def get_strata(prob_t_score, num_strata): 
    """
    Reports the strata given the probability score 
    
    Strata index starts at 1 
    """
    strata = np.arange(0, 1.0, 1.0/num_strata)
    return np.digitize(prob_t_score, strata, right=False)

#small toy test case 
get_strata(np.array([0.3, 0.6, 0.9]), num_strata)

array([2, 3, 5])

In [35]:
docid2strata = {}
for docid, prob_t_score in zip(docids, prob_t_inference): 
    strata_inferred = get_strata(prob_t_score, NUM_STRATA)
    docid2strata[docid] = strata_inferred

In [ ]:
# TODO: Put this in the same matching estimator 